# ActFlow 2b -- Compute the baseline decodability of Motor responses (LINDEX v. LMID and RINDEX v. RMID)
## Using ActFlow, one to one, via PCA-FC

## Use SVM classifications to decode hand-specific responses
## Using Ciric-style postprocessing

## Takuya Ito
#### 12/12/2018

In [58]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import multiprocessing as mp
import scipy.stats as stats
import nibabel as nib
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.api as sm
import sklearn.svm as svm
import statsmodels.sandbox.stats.multicomp as mc
import sklearn
from sklearn.feature_selection import f_classif
import seaborn as sns
import h5py
os.sys.path.append('glmScripts/')
import taskGLMPipeline as tgp
import statsmodels.api as sm

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [59]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

# 0.0 Define functions for loading data

In [60]:
def loadMotorResponses(subj,hand='Right'):
    
    hands = {'Left':[0,1],'Right':[2,3]}

    x = tgp.loadTaskTiming(subj,'ALL')
    stimIndex = np.asarray(x['stimIndex'])
    ind = np.where(stimIndex=='motorResponse')[0]
    
    datadir = basedir + 'data/postProcessing/hcpPostProcCiric/'
    h5f = h5py.File(datadir + subj + '_glmOutput_data.h5','r')
    data = h5f['taskRegression/ALL_24pXaCompCorXVolterra_taskReg_betas_canonical'][:].copy()
    data = data[:,ind].copy()
    h5f.close()
    
    # Isolate hand responses
    hand_ind = hands[hand]
    data = data[:,hand_ind]
    
    return data

def loadRSFCMapping(subj,roi):
    fcdir = '/projects3/SRActFlow/data/results/pcaFC/'
    filename = fcdir + 'TargetParcel' + str(roi) + '_pcaFC.h5'
    h5f = h5py.File(filename,'r')
    fcmapping = h5f[subj]['sourceToTargetMapping'][:].copy()
    h5f.close()
    return fcmapping


## Load masks
def loadMask(roi,dilated=True):
    maskdir = basedir + 'data/results/surfaceMasks/'
    if dilated:
        maskfile = maskdir + 'GlasserParcel' + str(roi) + '_dilated_10mm.dscalar.nii'
    else:
        maskfile = maskdir + 'GlasserParcel' + str(roi) + '.dscalar.nii'
    maskdata = np.squeeze(nib.load(maskfile).get_data())
    
    return maskdata
        

## 0.1 Load data for RH responses

In [61]:
# gsr = True
nResponses = 2
data_task = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadMotorResponses(subj,hand='Right')
    scount += 1

## 0.2 Generate actflow data for RH responses (Left S1)

In [62]:
roi_rh = 9 #left S1
target_rh_ind = np.where(glasser2==roi_rh)[0]
fcmapping_rh = np.zeros((len(glasser2),len(target_rh_ind)))

scount = 0
for subj in subjNums:
    fcmapping_rh[:,:] = fcmapping_rh[:,:] + loadRSFCMapping(subj,roi_rh)
    scount += 1

fcmapping_rh = np.divide(fcmapping_rh,len(subjNums))

In [63]:
roi_rh = 9
roi_lh = 189
actflow_data = np.zeros((len(glasser2),nResponses,len(subjNums),nParcels))

dilateLH = loadMask(roi_lh,dilated=True)
dilateRH = loadMask(roi_rh,dilated=True)
combinedDilated = dilateLH + dilateRH
source_space = np.where(combinedDilated==0)[0]
bad_rois = []
scount = 0
for subj in subjNums:
    print 'Subject', subj, '(', scount+1, '/', len(subjNums), ')'
    for roi in range(nParcels):
        source_ind = np.where(glasser2==roi+1)[0]
        source_ind = np.intersect1d(source_ind,source_space) # Make sure no vertices are within 10mm of the target
        if source_ind.shape[0]==0:
#             print 'This source has no vertices, skipping'
            bad_rois.append(roi)
            continue
        
        target_ind = np.where(glasser2==roi_rh)[0]
        # Right Finger 1
        actflow_data[target_ind,0,scount,roi] = np.dot(stats.zscore(data_task[source_ind,0,scount],axis=0),fcmapping_rh[source_ind,:])
        # Right Finger 2
        actflow_data[target_ind,1,scount,roi] = np.dot(stats.zscore(data_task[source_ind,1,scount],axis=0),fcmapping_rh[source_ind,:])

    scount += 1
    

Subject 013 ( 1 / 96 )
Subject 014 ( 2 / 96 )
Subject 016 ( 3 / 96 )
Subject 017 ( 4 / 96 )
Subject 018 ( 5 / 96 )
Subject 021 ( 6 / 96 )
Subject 023 ( 7 / 96 )
Subject 024 ( 8 / 96 )
Subject 026 ( 9 / 96 )
Subject 027 ( 10 / 96 )
Subject 028 ( 11 / 96 )
Subject 030 ( 12 / 96 )
Subject 031 ( 13 / 96 )
Subject 032 ( 14 / 96 )
Subject 033 ( 15 / 96 )
Subject 034 ( 16 / 96 )
Subject 035 ( 17 / 96 )
Subject 037 ( 18 / 96 )
Subject 038 ( 19 / 96 )
Subject 039 ( 20 / 96 )
Subject 040 ( 21 / 96 )
Subject 041 ( 22 / 96 )
Subject 042 ( 23 / 96 )
Subject 043 ( 24 / 96 )
Subject 045 ( 25 / 96 )
Subject 046 ( 26 / 96 )
Subject 047 ( 27 / 96 )
Subject 048 ( 28 / 96 )
Subject 049 ( 29 / 96 )
Subject 050 ( 30 / 96 )
Subject 053 ( 31 / 96 )
Subject 055 ( 32 / 96 )
Subject 056 ( 33 / 96 )
Subject 057 ( 34 / 96 )
Subject 058 ( 35 / 96 )
Subject 062 ( 36 / 96 )
Subject 063 ( 37 / 96 )
Subject 066 ( 38 / 96 )
Subject 067 ( 39 / 96 )
Subject 068 ( 40 / 96 )
Subject 069 ( 41 / 96 )
Subject 070 ( 42 / 96 )
S

# 1.0 Define functions for motor response decodings

In [64]:
from scipy.spatial.distance import cdist

def motorResponseDecodings(data, actflow_data, rois=[9], ncvs=1, nproc=5):
    """
    Run an across-subject classification
    Decode responses on each hand separately from CPRO data
    """

    nSubjs = data.shape[2]
    stats = np.zeros((len(rois),nSubjs*2))
    
    nfing = data.shape[1]

    nsamples = nSubjs * nfing

    # Label array for supervised learning
    labels = np.tile(range(nfing),nSubjs)
    subjarray = np.repeat(range(nSubjs),nfing)

    # Run SVM classifications on network-level activation patterns across subjects
    roicount = 0
    for roi in rois:
        roi_ind = np.where(glasser2==roi)[0]
        nfeatures = len(roi_ind)
        roi_ind.shape = (len(roi_ind),1)       

        svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
        actflow_svm_mat = np.zeros((nsamples,roi_ind.shape[0]))
        samplecount = 0
        scount = 0
        for subj in range(len(subjNums)):
            roidata = np.squeeze(data[roi_ind,:,scount])
            actflow_roidata = np.squeeze(actflow_data[roi_ind,:,scount])
            svm_mat[samplecount:(samplecount+nfing),:] = roidata.T
            actflow_svm_mat[samplecount:(samplecount+nfing),:] = actflow_roidata.T

            scount += 1
            samplecount += nfing

#             # Spatially demean matrix across features
#             samplemean = np.mean(svm_mat,axis=1)
#             samplemean.shape = (len(samplemean),1)
#             svm_mat = svm_mat - samplemean

        scores = randomSplitLOOBaselineCV(ncvs, svm_mat, actflow_svm_mat, labels, subjarray, nproc=nproc)
        stats[roicount,:] = scores
        roicount += 1
        
    return stats

def randomSplitLOOBaselineCV(ncvs, svm_mat, actflow_svm_mat, labels, subjarray, nproc=5):
    """
    Runs cross validation for an across-subject SVM analysis
    """
    
    ntasks = len(np.unique(labels))
    nsamples = svm_mat.shape[0]
    nsubjs = nsamples/ntasks

    subjects = np.unique(subjarray)
    indices = np.arange(nsamples)
    
    numsubjs_perfold = 16
    if nsubjs%numsubjs_perfold!=0: 
        raise Exception("Error: Folds don't match number of subjects")
        
    nfolds = nsubjs/numsubjs_perfold
    subj_array_folds = subjarray.copy()
    
    inputs = [] 
    
    for fold in range(nfolds):
        test_subjs = np.random.choice(subj_array_folds,numsubjs_perfold,replace=False)
        train_subjs_all = np.delete(subjects,test_subjs)
        for cv in range(ncvs):
            # Randomly sample half of train set subjects for each cv (CV bootstrapping)
            train_subjs = np.random.choice(train_subjs_all,
                                         int(np.floor(len(train_subjs_all)*(4.0))),
                                         replace=True)

            train_ind = []
            for subj in train_subjs:
                train_ind.extend(np.where(subjarray==subj)[0])

            test_ind = []
            for subj in test_subjs:
                test_ind.extend(np.where(subjarray==subj)[0])
            
            train_ind = np.asarray(train_ind)
            test_ind = np.asarray(test_ind)

            trainset = actflow_svm_mat[train_ind,:]
            testset = svm_mat[test_ind,:]

            # Normalize trainset and testset
            trainmean = np.mean(actflow_svm_mat[train_ind,:],axis=0)
            trainmean.shape = (1,len(trainmean))
            trainstd = np.std(actflow_svm_mat[train_ind,:],axis=0)
            trainstd.shape = (1,len(trainstd))
            
            # Normalize trainset and testset
            testmean = np.mean(svm_mat[train_ind,:],axis=0)
            testmean.shape = (1,len(testmean))
            teststd = np.std(svm_mat[train_ind,:],axis=0)
            teststd.shape = (1,len(teststd))

            trainset = np.divide((trainset - trainmean),trainstd)
            testset = np.divide((testset - testmean),teststd)

            ######## FEATURE SELECTION
            ## Feature selection and downsampling
            trainlabels = labels[train_ind]
            testlabels = labels[test_ind]
            unique_labels = np.unique(labels)
            feat1_labs = np.where(trainlabels==unique_labels[0])[0]
            feat2_labs = np.where(trainlabels==unique_labels[1])[0]
            # Perform t-test
            t, p = stats.ttest_rel(trainset[feat1_labs,:],trainset[feat2_labs,:],axis=0)
            h0, qs = mc.fdrcorrection0(p)
#             h0 = p<0.1
            # Construct feature masks
            feat1_mask = np.multiply(np.multiply(t,t>0),h0).astype(bool)
            feat2_mask = np.multiply(np.multiply(t,t<0),h0).astype(bool)
            feat1_mask = np.multiply(t,t>0).astype(bool)
            feat2_mask = np.multiply(t,t<0).astype(bool)
            # Downsample training set into original vertices into 2 ROI signals
            trainset_downsampled = np.zeros((trainset.shape[0],2))
            trainset_downsampled[:,0] = np.nanmean(trainset[:,feat1_mask],axis=1)
            trainset_downsampled[:,1] = np.nanmean(trainset[:,feat2_mask],axis=1)
#             trainset_downsampled = trainset[:,h0]
            # Downsample test set into original vertices
            testset_downsampled = np.zeros((testset.shape[0],2))
            testset_downsampled[:,0] = np.nanmean(testset[:,feat1_mask],axis=1)
            testset_downsampled[:,1] = np.nanmean(testset[:,feat2_mask],axis=1)
#             testset_downsampled = testset[:,h0]
            
            if np.sum(feat1_mask)==0 or np.sum(feat2_mask==0):
                inputs.append((trainset,testset,trainlabels,testlabels))
            else:
                inputs.append((trainset_downsampled,testset_downsampled,trainlabels,testlabels))

#             inputs.append((trainset,testset,labels[train_ind],labels[test_ind]))         
        subj_array_folds = np.delete(subj_array_folds,test_subjs)
        
    pool = mp.Pool(processes=nproc)
    scores = pool.map_async(_decoding,inputs).get()
    pool.close()
    pool.join()
    
#     subj_acc = np.zeros((len(subjects),))
#     scount = 0
#     i = 0
#     for subj in subjects:
#         subjmean = []
#         for cv in range(ncvs):
#             subjmean.append(scores[i])
#             i += 1
        
#         subj_acc[scount] = np.mean(subjmean)
        
#         scount += 1

#     return subj_acc

    acc = []
    for score in scores:
        acc.extend(score)
    return acc

def _decoding((trainset,testset,trainlabels,testlabels)):

#     clf = sklearn.linear_model.LogisticRegression()
    clf = svm.SVC(C=1.0, kernel='linear')

    clf.fit(trainset,trainlabels)
    predictions = clf.predict(testset)
    acc = predictions==testlabels
    
#     unique_cond = np.unique(trainlabels)
#     rdm = np.zeros((len(unique_cond),len(unique_cond)))
#     acc = []
#     for cond1 in unique_cond:
#         mismatches = []
#         prototype_ind = np.where(trainlabels==cond1)[0]
#         prototype = np.mean(trainset[prototype_ind,:],axis=0)
#         for cond2 in unique_cond:
#             test_ind = np.where(testlabels==cond2)[0]
#             test = np.mean(testset[test_ind,:],axis=0)
#             if cond1 == cond2: 
#                 correct = stats.pearsonr(prototype,test)[0]
#             else:
#                 mismatches.append(stats.pearsonr(prototype,test)[0])
        
#         if correct > np.max(mismatches): 
#             acc.append(1.0)
#         else:
#             acc.append(0.0)
    
    return acc

## 1.1 Run across subject decoding on right-hand motor responses

In [65]:
nproc = 20
ncvs = 1

rois = np.where(networkdef==networkmappings['smn'])[0] + 1
rois = [9] # Left S1

sourceROIs = []
sourceROIs.extend(np.where(networkdef==networkmappings['fpn'])[0])
sourceROIs.extend(np.where(networkdef==networkmappings['dan'])[0])
sourceROIs.extend(np.where(networkdef==networkmappings['con'])[0])

distances_baseline_rh = np.zeros((nParcels,len(subjNums)*nResponses))
for roi in sourceROIs:
    print 'Decoding roi', roi
    if roi in bad_rois: 
        distances_baseline_rh.append(np.zeros((1,len(subjNums)*2.0)))
        continue

    distances_baseline_rh[roi,:] = motorResponseDecodings(data_task, 
                                                          actflow_data[:,:,:,roi],
                                                          rois=rois, ncvs=ncvs, nproc=nproc)
    

Decoding roi 13
Decoding roi 14
Decoding roi 28
Decoding roi 62
Decoding roi 72
Decoding roi 76
Decoding roi 79
Decoding roi 81
Decoding roi 82
Decoding roi 84
Decoding roi 88
Decoding roi 90
Decoding roi 91
Decoding roi 96
Decoding roi 97
Decoding roi 110
Decoding roi 132
Decoding roi 143
Decoding roi 144
Decoding roi 148
Decoding roi 169
Decoding roi 170
Decoding roi 193
Decoding roi 194
Decoding roi 208
Decoding roi 237
Decoding roi 241
Decoding roi 242
Decoding roi 252
Decoding roi 253
Decoding roi 256
Decoding roi 259
Decoding roi 260
Decoding roi 262
Decoding roi 264
Decoding roi 268
Decoding roi 270
Decoding roi 271
Decoding roi 272
Decoding roi 276
Decoding roi 277
Decoding roi 290
Decoding roi 312
Decoding roi 323
Decoding roi 324
Decoding roi 328
Decoding roi 341
Decoding roi 349
Decoding roi 350
Decoding roi 356
Decoding roi 10
Decoding roi 45
Decoding roi 49
Decoding roi 94
Decoding roi 95
Decoding roi 115
Decoding roi 116
Decoding roi 126
Decoding roi 135
Decoding roi 136


## 1.2 Compute statistics

In [66]:
statistics_rh = np.zeros((distances_baseline_rh.shape[0],3))
for roicount in range(distances_baseline_rh.shape[0]):
    ntrials = len(subjNums)*2
    p = stats.binom_test(np.mean(distances_baseline_rh[roicount,:])*ntrials,n=ntrials,p=0.5)
    if np.mean(distances_baseline_rh[roicount,:])>0.5:
        p = p/2.0
    else:
        p = 1.0-p/2.0

    statistics_rh[roicount,0] = np.mean(distances_baseline_rh[roicount,:])
    statistics_rh[roicount,1] = p

# rois_notSMN = np.where(networkdef!=networkmappings['smn'])[0]
rois_testing = []
rois_testing.extend(np.where(networkdef==networkmappings['fpn'])[0])
rois_testing.extend(np.where(networkdef==networkmappings['con'])[0])
rois_testing.extend(np.where(networkdef==networkmappings['dan'])[0])

h0, qs = mc.fdrcorrection0(statistics_rh[rois_testing,1])

statistics_rh[:,1] = 1.0
i = 0
for roi in rois_testing:
    statistics_rh[roi,1] = qs[i]
    statistics_rh[roi,2] = h0[i]*statistics_rh[roi,0]
    i += 1
        
nSignificant = np.sum(statistics_rh[:,1] < 0.05)
print 'Number of significant transfers:', nSignificant

if nSignificant>0:
    sig_ind = np.where(statistics_rh[:,1]<0.05)[0]
    for ind in sig_ind:
        print '\tSignificant parcel:', ind+1
        print '\tAccuracy:', statistics_rh[ind,0]
        print '\tNetwork:', networkdef[ind]
print 'Network mappings:', networkmappings

Number of significant transfers: 0
Network mappings: {'vis2': 2, 'lan': 6, 'vis1': 1, 'none2': 12, 'none1': 11, 'dan': 5, 'aud': 8, 'pmulti': 10, 'fpn': 7, 'dmn': 9, 'smn': 3, 'con': 4}


# Generate ActFlow predictions for the Left Hand (Right S1)

## 0.1 Load data for LH responses

In [67]:
# gsr = True
nResponses = 2
data_task = np.zeros((len(glasser2),nResponses,len(subjNums)))

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = loadMotorResponses(subj,hand='Left')
    scount += 1

## 0.2 Generate actflow data for LH responses (Right S1)

In [68]:
roi_lh = 189 #right S1
target_lh_ind = np.where(glasser2==roi_lh)[0]
fcmapping_lh = np.zeros((len(glasser2),len(target_lh_ind)))

scount = 0
for subj in subjNums:
    fcmapping_lh[:,:] = fcmapping_lh[:,:] + loadRSFCMapping(subj,roi_lh)
    scount += 1

fcmapping_lh = np.divide(fcmapping_lh,len(subjNums))

In [69]:
roi_rh = 9
roi_lh = 189
actflow_data = np.zeros((len(glasser2),nResponses,len(subjNums),nParcels))

dilateLH = loadMask(roi_lh,dilated=True)
dilateRH = loadMask(roi_rh,dilated=True)
combinedDilated = dilateLH + dilateRH
source_space = np.where(combinedDilated==0)[0]
bad_rois = []
scount = 0
for subj in subjNums:
    print 'Subject', subj, '(', scount+1, '/', len(subjNums), ')'
    for roi in range(nParcels):
        source_ind = np.where(glasser2==roi+1)[0]
        source_ind = np.intersect1d(source_ind,source_space) # Make sure no vertices are within 10mm of the target
        if source_ind.shape[0]==0:
#             print 'This source has no vertices, skipping'
            bad_rois.append(roi)
            continue    
        
        target_ind = np.where(glasser2==roi_lh)[0]
        # Right Finger 1
        actflow_data[target_ind,0,scount,roi] = np.dot(stats.zscore(data_task[source_ind,0,scount],axis=0),fcmapping_lh[source_ind,:])
        # Right Finger 2
        actflow_data[target_ind,1,scount,roi] = np.dot(stats.zscore(data_task[source_ind,1,scount],axis=0),fcmapping_lh[source_ind,:])

    scount += 1
    

Subject 013 ( 1 / 96 )
Subject 014 ( 2 / 96 )
Subject 016 ( 3 / 96 )
Subject 017 ( 4 / 96 )
Subject 018 ( 5 / 96 )
Subject 021 ( 6 / 96 )
Subject 023 ( 7 / 96 )
Subject 024 ( 8 / 96 )
Subject 026 ( 9 / 96 )
Subject 027 ( 10 / 96 )
Subject 028 ( 11 / 96 )
Subject 030 ( 12 / 96 )
Subject 031 ( 13 / 96 )
Subject 032 ( 14 / 96 )
Subject 033 ( 15 / 96 )
Subject 034 ( 16 / 96 )
Subject 035 ( 17 / 96 )
Subject 037 ( 18 / 96 )
Subject 038 ( 19 / 96 )
Subject 039 ( 20 / 96 )
Subject 040 ( 21 / 96 )
Subject 041 ( 22 / 96 )
Subject 042 ( 23 / 96 )
Subject 043 ( 24 / 96 )
Subject 045 ( 25 / 96 )
Subject 046 ( 26 / 96 )
Subject 047 ( 27 / 96 )
Subject 048 ( 28 / 96 )
Subject 049 ( 29 / 96 )
Subject 050 ( 30 / 96 )
Subject 053 ( 31 / 96 )
Subject 055 ( 32 / 96 )
Subject 056 ( 33 / 96 )
Subject 057 ( 34 / 96 )
Subject 058 ( 35 / 96 )
Subject 062 ( 36 / 96 )
Subject 063 ( 37 / 96 )
Subject 066 ( 38 / 96 )
Subject 067 ( 39 / 96 )
Subject 068 ( 40 / 96 )
Subject 069 ( 41 / 96 )
Subject 070 ( 42 / 96 )
S

## 1.1 Run across subject decoding on left-hand motor responses

In [70]:
nproc = 20
ncvs = 1

rois = np.where(networkdef==networkmappings['smn'])[0] + 1
rois = [189] # Right S1

sourceROIs = []
sourceROIs.extend(np.where(networkdef==networkmappings['fpn'])[0])
sourceROIs.extend(np.where(networkdef==networkmappings['dan'])[0])
sourceROIs.extend(np.where(networkdef==networkmappings['con'])[0])

distances_baseline_lh = np.zeros((nParcels,len(subjNums)*nResponses))
for roi in sourceROIs:
    print 'Decoding roi', roi
    if roi in bad_rois: 
        distances_baseline_lh.append(np.zeros((1,len(subjNums)*2.0)))
        continue

    distances_baseline_lh[roi,:] = motorResponseDecodings(data_task,
                                                          actflow_data[:,:,:,roi],
                                                          rois=rois, ncvs=ncvs, nproc=nproc)
    

Decoding roi 13
Decoding roi 14
Decoding roi 28
Decoding roi 62
Decoding roi 72
Decoding roi 76
Decoding roi 79
Decoding roi 81
Decoding roi 82
Decoding roi 84
Decoding roi 88
Decoding roi 90
Decoding roi 91
Decoding roi 96
Decoding roi 97
Decoding roi 110
Decoding roi 132
Decoding roi 143
Decoding roi 144
Decoding roi 148
Decoding roi 169
Decoding roi 170
Decoding roi 193
Decoding roi 194
Decoding roi 208
Decoding roi 237
Decoding roi 241
Decoding roi 242
Decoding roi 252
Decoding roi 253
Decoding roi 256
Decoding roi 259
Decoding roi 260
Decoding roi 262
Decoding roi 264
Decoding roi 268
Decoding roi 270
Decoding roi 271
Decoding roi 272
Decoding roi 276
Decoding roi 277
Decoding roi 290
Decoding roi 312
Decoding roi 323
Decoding roi 324
Decoding roi 328
Decoding roi 341
Decoding roi 349
Decoding roi 350
Decoding roi 356
Decoding roi 10
Decoding roi 45
Decoding roi 49
Decoding roi 94
Decoding roi 95
Decoding roi 115
Decoding roi 116
Decoding roi 126
Decoding roi 135
Decoding roi 136


## 1.2 Compute statistics

In [71]:
statistics_lh = np.zeros((distances_baseline_lh.shape[0],3))
for roicount in range(distances_baseline_lh.shape[0]):
    ntrials = len(subjNums)*2
    p = stats.binom_test(np.mean(distances_baseline_lh[roicount,:])*ntrials,n=ntrials,p=0.5)
    if np.mean(distances_baseline_lh[roicount,:])>0.5:
        p = p/2.0
    else:
        p = 1.0-p/2.0

    statistics_lh[roicount,0] = np.mean(distances_baseline_lh[roicount,:])
    statistics_lh[roicount,1] = p

# rois_notSMN = np.where(networkdef!=networkmappings['smn'])[0]
rois_testing = []
rois_testing.extend(np.where(networkdef==networkmappings['fpn'])[0])
rois_testing.extend(np.where(networkdef==networkmappings['con'])[0])
rois_testing.extend(np.where(networkdef==networkmappings['dan'])[0])

h0, qs = mc.fdrcorrection0(statistics_lh[rois_testing,1])

statistics_lh[:,1] = 1.0
i = 0
for roi in rois_testing:
    statistics_lh[roi,1] = qs[i]
    statistics_lh[roi,2] = h0[i]*statistics_lh[roi,0]
    i += 1
        
nSignificant = np.sum(statistics_lh[:,1] < 0.05)
print 'Number of significant transfers:', nSignificant

if nSignificant>0:
    sig_ind = np.where(statistics_lh[:,1]<0.05)[0]
    for ind in sig_ind:
        print '\tSignificant parcel:', ind+1
        print '\tAccuracy:', statistics_lh[ind,0]
        print '\tNetwork:', networkdef[ind]
print 'Network mappings:', networkmappings

Number of significant transfers: 0
Network mappings: {'vis2': 2, 'lan': 6, 'vis1': 1, 'none2': 12, 'none1': 11, 'dan': 5, 'aud': 8, 'pmulti': 10, 'fpn': 7, 'dmn': 9, 'smn': 3, 'con': 4}


In [72]:
print np.max(statistics_lh[:,0]), statistics_lh[:,0].argmax()
print np.max(statistics_rh[:,0]), statistics_rh[:,0].argmax()

0.567708333333 56
0.583333333333 143
